## 문제 6

**Kaggle 형** train_prob.csv로 문제 target을 예측하는 모델을 만들고, 

test_prob.csv에 대한 target 예측하여 다음과 같은 형식의 answer6.csv를 만들어라.

id, target

0, 6.9

5, 7.8

...


**평가지표**

$RMSE(Y, \hat{Y}) = \sqrt{\frac{1}{n}\sum^{n}_{i=1}(y_i-\hat{y_i})^2}$

**강사: 멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)**

# 모수적 모델 특히 선형모델에서는 가변수를 만들때 drop='first'가 꼭 필요합니다

**※ 의사 결정 나무와 같은 비모수 모델에서는 drop='first'는 불필요하여 상관이 없습니다.**

In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys
import xgboost as xgb

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
xgboost 0.80


In [2]:
df_train = pd.read_csv('train_prob.csv', index_col=['id'])
df_test = pd.read_csv('test_prob.csv', index_col=['id'])
df_ans = pd.read_csv('test_prob_ans.csv', index_col=['id'])

In [3]:
# 처리 과정에 필요하 내용들을 list 형태로 구성합니다.
repl_list = [
    ('cat3', {"B": "C"}, [83634, 147361, 9005]),
    ('cat4', {"A": "B", "D": "B"}, [239397, 603]), 
    ('cat6', {'D': 'A', 'E': 'B', 'G': 'C', 'H': 'B', 'I': 'A'}, [234203, 5145, 652]),
    ('cat7', {'A': 'B', 'C': 'B', 'F': 'D', 'I': 'B'}, [4606, 19784, 214027, 1583]),
    ('cat8', {'B': 'G', 'F': 'E'}, [30338, 96743, 2953, 76085, 33881]),
    ('cat9', {'C': 'H', 'D': 'B', 'E': 'L'}, [10678, 2846, 85944, 8320, 19987, 40070, 5501, 16743, 33793, 7819, 3331, 4968])
]
# 반복문 처리 내용들을 수행합니다.
for c, d, cnt in repl_list:
    print(c, d, cnt)
    # 치환후 내용을 s_repl에 저장합니다
    s_repl = df_train[c].replace(d)
    # 치환결과를 확인합니다.
    if not (s_repl.nunique() == len(cnt) and (s_repl.value_counts().sort_index() == cnt).all()):
        print("Error", c)
        break
    df_train[c] = s_repl
     # 일부러 cat9에 test에만 등장하는 수준을 만듭니다.
    if c != 'cat9':
        df_test[c] = df_test[c].replace(d)

cat3 {'B': 'C'} [83634, 147361, 9005]
cat4 {'A': 'B', 'D': 'B'} [239397, 603]
cat6 {'D': 'A', 'E': 'B', 'G': 'C', 'H': 'B', 'I': 'A'} [234203, 5145, 652]
cat7 {'A': 'B', 'C': 'B', 'F': 'D', 'I': 'B'} [4606, 19784, 214027, 1583]
cat8 {'B': 'G', 'F': 'E'} [30338, 96743, 2953, 76085, 33881]
cat9 {'C': 'H', 'D': 'B', 'E': 'L'} [10678, 2846, 85944, 8320, 19987, 40070, 5501, 16743, 33793, 7819, 3331, 4968]


In [4]:
q = [i for i in np.arange(0, 1.01, 0.01)]
# 나머지 변수에 대해서도 해당 파생 변수를 만들어 줍니다.
for i in range(0, 14):
    col = 'cont{}'.format(i)
    q_val = df_train[col].quantile(q)
    q_val.iloc[[0, -1]] = [-np.inf, np.inf]
    q_range = pd.cut(df_train[col], bins=q_val)
    q_mean = df_train.groupby(q_range)['target'].mean()
    df_train[col + '_q'] = q_range.map(q_mean).astype('float')
    df_test[col + '_q'] = pd.cut(df_test[col], bins=q_val).map(q_mean).astype('float')

In [5]:
# 공통
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import ShuffleSplit, KFold 

cv = KFold(n_splits=5, random_state=123)
# train(80%)/test(20%) 한 번으로 검증합니다. XGB, RF등 오래 걸리는 모델을 위해 사용합니다.
ss = ShuffleSplit(n_splits=1, train_size=0.8, random_state=123)

df_ans = pd.read_csv('test_prob_ans.csv', index_col='id')
X = df_test.columns.tolist() + ['cont{}_q'.format(i) for i in range(14)]

cat_cols = ['cat{}'.format(i) for i in range(10)]
cont_cols = ['cont{}'.format(i) for i in range(14)]
cont_q_cols = ['cont{}_q'.format(i) for i in range(14)]

# 위에서 발생한 leak을 바로 잡아 교차검증을 합니다.
q = [i for i in np.arange(0, 1.01, 0.01)]
def eval_model(model, sp):
    score_train, score = list(), list()
    for train_idx, test_idx in sp.split(df_train):
        df_cv_train, df_cv_test = df_train.iloc[train_idx].copy(), df_train.iloc[test_idx].copy()
        # 검증셋에서 train으로 파생변수를 만들고
        # 검증셋의 test(겹외셋)에 검증셋의 train으로 만든 통계값(mean)을 반영합니다.
        for i in range(0, 14):
            col = 'cont{}'.format(i)
            qt = df_cv_train[col].quantile(q)
            qt.iloc[[0, -1]] = [-np.inf, np.inf]
            q_range = pd.cut(df_cv_train[col], bins=qt)
            q_mean = df_cv_train.groupby(q_range)['target'].mean()
            df_cv_train[col + '_q'] = q_range.map(q_mean).astype('float')
            df_cv_test[col + '_q'] = pd.cut(df_cv_test[col], bins=qt).map(q_mean).astype('float')
        model.fit(df_cv_train[X], df_cv_train['target'])
        score_train.append(-(mean_squared_error(df_cv_train['target'], model.predict(df_cv_train[X]))) ** 0.5)
        score.append(-(mean_squared_error(df_cv_test['target'], model.predict(df_cv_test[X]))) ** 0.5)
    return score_train, score

def choose_model(model):
    model.fit(df_train[X], df_train['target'])
    prd = model.predict(df_test[X])
    pd.DataFrame({
        'id': df_test.index.values,
        'target': prd
    }).to_csv('answer6.csv', index = None)
    return prd 

In [6]:
# Baseline 모델을 만들어 봅니다.
# handle_unknown을 ignore로 하여 등장하지 않은 수준에 대해 에러가 발생하지 않도록 합니다.
# 이 경우 해당 변수의 가변수 모든 값은 0이 됩니다.
# handle_unknown = 'ignore'와 drop='first'를 동시에 설정할 수 없습니다.
# drop='first'를 생략하면 어떤 문제가 발생하는지 확인하기 위해 drop='first'를 제외합니다.
from sklearn.linear_model import LinearRegression

ct = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown='ignore'), cat_cols),
    ('std', StandardScaler(), cont_cols),
])
reg_lr = make_pipeline(
    ct, 
    LinearRegression()
)
result = eval_model(reg_lr, cv)
np.mean(result[1]), np.std(result[1]), np.mean(result[0])

(-0.8632710692832337, 0.0029506483961325154, -0.8630391361652678)

In [7]:
prd = choose_model(reg_lr)
# 자가 채점을 해봅니다.
# 검증시에는 문제가 보이지 않지만
# 제출한 test 결과에는 심한 결합이 보입니다.
# 변수마다 하나의 가변수를 제외하지 않아, 완전한 다중공선성에 의한
# 모델의 불안정성이 심하게되어 문제를 일으킨 것입니다.
mean_squared_error(df_ans['target'], prd) ** 0.5

3514777007.3882246

In [8]:
# 회귀 계수에 큰 값이 채워져 있습니다.
# 불안정한 모델이 되었습니다.
reg_lr.fit(df_train[X], df_train['target'])
reg_lr[1].coef_

array([ 8.88603061e+09,  8.88603061e+09, -8.38064646e+09, -8.38064646e+09,
       -9.88098395e+09, -9.88098395e+09,  8.58976592e+09,  8.58976592e+09,
        8.58976592e+09,  1.22720922e+11,  1.22720922e+11,  7.02048573e+09,
        7.02048573e+09,  7.02048573e+09,  7.02048573e+09,  4.94934768e+10,
        4.94934768e+10,  4.94934768e+10,  1.25524042e+11,  1.25524042e+11,
        1.25524042e+11,  1.25524042e+11, -4.84038575e+09, -4.84038575e+09,
       -4.84038575e+09, -4.84038575e+09, -4.84038575e+09,  9.68634328e+10,
        9.68634328e+10,  9.68634328e+10,  9.68634328e+10,  9.68634328e+10,
        9.68634328e+10,  9.68634328e+10,  9.68634328e+10,  9.68634328e+10,
        9.68634328e+10,  9.68634328e+10,  9.68634328e+10, -5.82885742e-02,
        3.45573425e-02,  2.10189819e-03, -5.52368164e-03, -5.50842285e-03,
       -5.46493530e-02,  2.36263275e-02,  1.55296326e-02,  6.80694580e-02,
        2.72369385e-02, -2.11944580e-02,  5.17578125e-02,  1.56860352e-02,
       -5.84411621e-03])

# 진단 및 해결

In [9]:
# 따라서 Linear과 같은 모수적 모델에서는 모델의 안정성을 위해, 
# drop='first'를 반드시 해주어야 합니다. 
# Test만 등장하는 범주값을 제거해 줍니다.

# Test에만 등장하는 범주값을 봅니다.

pd.concat([
    df_train[[k[0] for k in repl_list]].apply(lambda x: set(x.unique())).rename('train_level'),
    df_test[[k[0] for k in repl_list]].apply(lambda x: set(x.unique())).rename('test_level')
], axis=1).apply(lambda x: x['test_level'] - x['train_level'], axis=1)

cat3           {}
cat4           {}
cat6           {}
cat7           {}
cat8           {}
cat9    {E, C, D}
dtype: object

In [10]:
# 미등장 수준을 가장 자주 등장하는 범주값으로 바꿉니다.
df_test['cat9'] = df_test['cat9'].replace({'D': np.nan, 'E': np.nan, 'C': np.nan})
df_test['cat9'] = df_test['cat9'].fillna(df_train['cat9'].mode().iloc[0])

In [11]:
# drop='first' 설정을 해줍니다.
from sklearn.linear_model import LinearRegression
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(drop='first'), ['cat{}'.format(i) for i in range(10)]),
    ('pt', 'passthrough', ['cont{}'.format(i) for i in range(14)])
])
X_lr = ['cat{}'.format(i) for i in range(10)] + ['cont{}'.format(i) for i in range(14)]
reg_lr = make_pipeline(ct, LinearRegression())
scores_train, scores = eval_model(reg_lr, cv)
np.mean(scores), np.std(scores), np.mean(scores_train)

(-0.8632456423386847, 0.0029474142614306633, -0.8630025788522048)

In [12]:
# test에 등장하지 않은 변수값을 train에 등장하는 범주값으로 처리해주었고,
# drop='first'를 통해 완전한 다중공선성이 발생하지 않도록 했습니다.
# 문제가 해결됩니다.
prd = choose_model(reg_lr)
mean_squared_error(df_ans['target'], prd) ** 0.5

0.8657268588416211

In [13]:
# 회귀 계수를 출력해보면 지나치게 큰 계수가 나왔던 문제가 완전히 해결됩니다.
reg_lr.fit(df_train[X], df_train['target'])
reg_lr[1].coef_

array([-0.14737833,  0.16983335,  0.24147158,  0.10693804,  0.03994362,
        0.15713566,  0.03681214, -0.17486971,  0.03074087,  0.21519861,
        0.36870383, -0.00597853, -0.05547379, -0.05870304, -0.00092302,
        0.05854559,  0.07181603, -0.10018803,  0.03508515,  0.13365193,
        0.1647826 ,  0.08022279, -0.00404722,  0.02865399,  0.09003857,
        0.21092791,  0.09276305,  0.09655687,  0.07017521, -0.28413913,
        0.14685393,  0.01069457, -0.02323074, -0.02707619, -0.23571329,
        0.12262252,  0.07588697,  0.30830852,  0.13373422, -0.10509757,
        0.22408617,  0.0712212 , -0.02577207])